In [1]:
import time
import pandas as pd
from datetime import date
from datetime import datetime
from datetime import timedelta
import win32com.client as win32
import os
start_time = time.time()

In [2]:
# Archivos
d1 = '9030-D1.XLSX'
d5 = '9030-D5.XLSX'
d6 = '9030-D6.XLSX'
d7 = '9030-D7.XLSX'
TU = 'TU 15-09.XLSX'
d9030_hoy = '9030-DESCARGA.XLSX'
d9030_anterior = '9030-DESCARGA-14-09-2023.XLSX'

In [3]:
# Leer 9030-DESCARGA del dia de hoy
Columnas_9030 = ['Documento', 'Número de posición', 'Pedido de compras', 'Posición de pedido', 'Orden de servicio',
                 'Operación de orden de servicio', 'Documento ERP', 'Posición de documento de ERP', 'Aviso de entrega', 
                 'Prod.', 'Descripción de producto', 'Ctd.', 'Unidad medida', 'Propietario del stock', 
                 'Status de entrada en stock','Status de la entrada de mercancías', 'Persona autorizada a disponer']

df1 = pd.read_excel(d9030_hoy, dtype = 'string', usecols = Columnas_9030)

# Vaciar valores columnas Fecha y Hora
df1.loc[: , ['Status de entrada en stock', 'Status de la entrada de mercancías']] = pd.NA

# Vaciar Ordenes y Operaciones con OS = Vacío
df1.loc[pd.isna(df1['Orden de servicio']) , ['Orden de servicio', 'Operación de orden de servicio']] = pd.NA

C:\Users\pablo.gutierrez\AppData\Local\Temp\ipykernel_13420\1327049712.py:10: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df1.loc[: , ['Status de entrada en stock', 'Status de la entrada de mercancías']] = pd.NA


In [4]:
# Crear Columnas
df1['ERP+POS+MAT+CTD'] = df1['Documento ERP'] + df1['Posición de documento de ERP'] + df1['Prod.'] + df1['Ctd.']
df1['OS+OP+MAT+CTD'] = df1['Orden de servicio'] + df1['Operación de orden de servicio'] + df1['Prod.'] + df1['Ctd.']
df1['D+I'] =  df1['Documento'] + df1['Número de posición']

In [5]:
# Identificador Único
df1['Identificador Unico'] = df1.index.values.tolist()

In [6]:
# Cruce UMP.SUP y UMP entre 9030 de hoy y el del dia anterior

# Leer 9030-DESCARGA dia anterior
df2 = pd.read_excel(d9030_anterior, dtype = 'string', usecols = ['Documento', 'Número de posición', 'UMP.SUP','UMP'])
df2['D+I'] = df2['Documento'] + df2['Número de posición']
df2.drop(labels = ['Documento', 'Número de posición'], axis = 1, inplace = True)

# Join, asi obtenemos UMP.SUP y UMP
df1.set_index('D+I', inplace = True)
df2.set_index('D+I', inplace = True)
df1 = df1.join(df2, how = 'left')
df1.reset_index(inplace = True)

In [7]:
# Aviso de Entrega
df1.loc[(pd.isna(df1['UMP'])) & (df1['Aviso de entrega'].str.contains('-Z')) , 'UMP'] = \
list(map(lambda x: str(x[0:8]) , df1.loc[(pd.isna(df1['UMP'])) & (df1['Aviso de entrega'].str.contains('-Z')) , 'Aviso de entrega']))

df1.loc[(pd.isna(df1['UMP.SUP'])) & (df1['Aviso de entrega'].str.contains('-Z')) , 'UMP.SUP'] = \
list(map(lambda x: str(x[0:8]) , df1.loc[(pd.isna(df1['UMP.SUP'])) & (df1['Aviso de entrega'].str.contains('-Z')) , 'Aviso de entrega']))

In [8]:
# D5 y D6
df3 = pd.read_excel(d5, usecols = ['Documento', 'Orden de servicio', 'Operación de orden de servicio'], dtype = 'string')
df4 = pd.read_excel(d6, usecols = ['Documento', 'Producto', 'Ctd.real dest.UMB', 'Un.manipulac.origen', 'UMp destino',
                                  'Fecha confirmación'], dtype = 'string')

In [9]:
# Obtener OS y OP en 9030-D6

# Eliminar Duplicados de Documento en df3
df3.drop_duplicates(subset = 'Documento', keep = 'first', inplace = True)
df3.set_index('Documento', inplace = True)
df4.set_index('Documento', inplace = True)
df4 = df4.join(df3, how = 'left')
df4.reset_index(inplace = True)
df4['OS+OP+MAT+CTD'] = df4['Orden de servicio'] + df4['Operación de orden de servicio'] +\
df4['Producto'] + df4['Ctd.real dest.UMB']

In [10]:
# Primer Cruce para buscar UMp destino

# Nuevo dataframe para realizar modificaciones
df5 = df4.loc[: , ['OS+OP+MAT+CTD', 'UMp destino', 'Fecha confirmación']]

# Eliminar celdas vacías
df5.dropna(subset = 'OS+OP+MAT+CTD', axis = 0, inplace = True)
df5.dropna(subset = 'UMp destino', axis = 0, inplace = True)
df5.dropna(subset = 'Fecha confirmación', axis = 0, inplace = True)

# Eliminar '00:00:00' columna Fecha confirmación
df5['Fecha confirmación'] = df5['Fecha confirmación'].str.replace(" 00:00:00","", regex = False)

# Fecha confirmación a formato a Datetime
df5['Fecha confirmación'] = pd.to_datetime(df5['Fecha confirmación'], errors = 'raise', format = "%Y/%m/%d", yearfirst = True)

# Ordenar Fecha confirmación por mas Reciente
df5.sort_values(by = 'Fecha confirmación', ascending = False, inplace = True)

# Eliminar Duplicados
df5.drop_duplicates(subset = 'OS+OP+MAT+CTD', keep = 'first', inplace = True)

# Eliminar columna Fecha confirmación
df5.drop(labels = 'Fecha confirmación', axis = 1, inplace = True)

# Join
df5.set_index('OS+OP+MAT+CTD', inplace = True)
df1.set_index('OS+OP+MAT+CTD', inplace = True)
df1 = df1.join(df5, how = 'left')
df1.sort_values(by = 'Identificador Unico', ascending = True, inplace = True)
df1.reset_index(inplace = True)

# Relleno celdas vacías UMP con UMp destino
df1.loc[pd.isna(df1['UMP']), 'UMP'] = df1.loc[pd.isna(df1['UMP']), 'UMp destino']

# Eliminar columna UMP.destino de df1
df1.drop(labels = 'UMp destino', axis = 1, inplace = True)

In [11]:
# Segundo Cruce UMp.Destino y Unidad de Manipulacion Origen

# Nuevo dataframe para realizar modificaciones
df6 = df4.loc[: , ['UMp destino', 'Fecha confirmación', 'Un.manipulac.origen']]

# Eliminar celdas vacías
df6.dropna(subset = 'Un.manipulac.origen', axis = 0, inplace = True)
df6.dropna(subset = 'UMp destino', axis = 0, inplace = True)
df6.dropna(subset = 'Fecha confirmación', axis = 0, inplace = True)

# Eliminar '00:00:00' columna Fecha confirmación
df6['Fecha confirmación'] = df6['Fecha confirmación'].str.replace(" 00:00:00","", regex = False)

# Fecha confirmación a formato a Datetime
df6['Fecha confirmación'] = pd.to_datetime(df6['Fecha confirmación'], errors = 'raise', format = "%Y/%m/%d", yearfirst = True)

# Ordenar Fecha confirmación por mas Reciente
df6.sort_values(by = 'Fecha confirmación', ascending = False, inplace = True)

# Eliminar Fecha de Confirmación
df6.drop(labels = 'Fecha confirmación', axis = 1, inplace = True)

In [12]:
# df7: Primero las 33*
df7 = df6.loc[df6['UMp destino'].str.startswith('33'), :]

# ELiminar duplicados de Contenedora
df7 = df7.drop_duplicates(subset = 'Un.manipulac.origen', keep = 'first')

# Join
df7.set_index('Un.manipulac.origen', inplace = True)
df1.set_index('UMP', inplace = True)
df1 = df1.join(df7, how = 'left')
df1.sort_values(by = 'Identificador Unico', ascending = True, inplace = True)
df1.reset_index(inplace = True)
df1.rename(columns = {'index': 'UMP'}, inplace = True)

# Relleno celdas vacías UMP.SUP con UMp destino
df1.loc[pd.isna(df1['UMP.SUP']), 'UMP.SUP'] = df1.loc[pd.isna(df1['UMP.SUP']), 'UMp destino']

# Eliminar columna UMP.destino de df1
df1.drop(labels = 'UMp destino', axis = 1, inplace = True)

In [13]:
# df8: Distinto a las 33*
df8 = df6.loc[(df6['UMp destino'].str.startswith('33')) == False , :] 

# ELiminar duplicados de Contenedora
df8 = df8.drop_duplicates(subset = 'Un.manipulac.origen', keep = 'first')

# Join
df8.set_index('Un.manipulac.origen', inplace = True)
df1.set_index('UMP', inplace = True)
df1 = df1.join(df8, how = 'left')
df1.sort_values(by = 'Identificador Unico', ascending = True, inplace = True)
df1.reset_index(inplace = True)
df1.rename(columns = {'index': 'UMP'}, inplace = True)

# Relleno celdas vacías UMP.SUP con UMp destino
df1.loc[pd.isna(df1['UMP.SUP']), 'UMP.SUP'] = df1.loc[pd.isna(df1['UMP.SUP']), 'UMp destino']

# Eliminar columna UMP.destino de df1
df1.drop(labels = 'UMp destino', axis = 1, inplace = True)

In [14]:
# 9030-D7

# Leer archivo
df9 = pd.read_excel(d7, dtype = 'string', usecols = ['Entrega','Posición','Material','Cantidad entrega', 'CASE'])

# Transformaciones a la columna Cantidad de entrega para el concatenado de ERP+POS+MAT+CTD
df9['Cantidad entrega'] = list(map(float, df9['Cantidad entrega']))
df9['Cantidad entrega'] = list(map(int, df9['Cantidad entrega']))
df9['Cantidad entrega'] = list(map(str, df9['Cantidad entrega']))

# Crear columna ERP+POS+MAT+CTD, eliminar duplicados
df9['ERP+POS+MAT+CTD'] = df9['Entrega'] + df9['Posición'] + df9['Material'] + df9['Cantidad entrega']
df9.drop_duplicates(subset = 'ERP+POS+MAT+CTD', keep = 'first', inplace = True)

# Eliminar columnas no necesarias para el cruce
df9.drop(labels = ['Entrega', 'Posición', 'Cantidad entrega', 'Material'], axis = 1, inplace = True)

# Join
df9.set_index('ERP+POS+MAT+CTD', inplace = True)
df1.set_index('ERP+POS+MAT+CTD', inplace = True)
df1 = df1.join(df9, how = 'left')
df1.sort_values(by = 'Identificador Unico', ascending = True, inplace = True)
df1.reset_index(inplace = True)

# Relleno Celdas Vacías UMP con Case
df1.loc[pd.isna(df1['UMP']), 'UMP'] = df1.loc[pd.isna(df1['UMP']), 'CASE']

# Eliminar Columna Case
df1.drop(labels = 'CASE', axis = 1, inplace = True)

In [15]:
# Volver a rellenar UMP.SUP con las 33* Primero y despues las distinta a 33*

# Join
df1.set_index('UMP', inplace = True)
df1 = df1.join(df7, how = 'left')
df1.sort_values(by = 'Identificador Unico', ascending = True, inplace = True)
df1.reset_index(inplace = True)
df1.rename(columns = {'index': 'UMP'}, inplace = True)

# Relleno celdas vacías UMP.SUP con UMp destino
df1.loc[pd.isna(df1['UMP.SUP']), 'UMP.SUP'] = df1.loc[pd.isna(df1['UMP.SUP']), 'UMp destino']

# Eliminar columna UMP.destino de df1
df1.drop(labels = 'UMp destino', axis = 1, inplace = True)

In [16]:
# Distinto a 33*

# Join
df1.set_index('UMP', inplace = True)
df1 = df1.join(df8, how = 'left')
df1.sort_values(by = 'Identificador Unico', ascending = True, inplace = True)
df1.reset_index(inplace = True)
df1.rename(columns = {'index': 'UMP'}, inplace = True)

# Relleno celdas vacías UMP.SUP con UMp destino
df1.loc[pd.isna(df1['UMP.SUP']), 'UMP.SUP'] = df1.loc[pd.isna(df1['UMP.SUP']), 'UMp destino']

# Eliminar columna UMP.destino de df1
df1.drop(labels = 'UMp destino', axis = 1, inplace = True)

In [17]:
# Si UMP empieza con 1000, entonces UMP.SUP = UMP
df1.loc[df1['UMP'].str.startswith('1000'), 'UMP.SUP'] = df1.loc[df1['UMP'].str.startswith('1000'), 'UMP']

In [18]:
# Cruce para Fecha y Hora usando d1

# Leer archivo
df10 = pd.read_excel(d1, dtype = 'string', usecols = ['Fecha de creación', 'Hora de creación', 'Un.manipulac.origen'])

# Eliminar '00:00:00' columna Fecha confirmación
df10['Fecha de creación'] = df10['Fecha de creación'].str.replace(" 00:00:00","", regex = False)

# Fecha de creación a formato a Datetime
df10['Fecha de creación'] = pd.to_datetime(df10['Fecha de creación'], errors = 'raise', format = "%Y/%m/%d", yearfirst = True)

# Ordenar por mas reciente
df10.sort_values(by = 'Fecha de creación', ascending = False, inplace = True)

# Fecha a string
df10['Fecha de creación'] = df10['Fecha de creación'].dt.strftime("%d/%m/%Y")

# Eliminar NA para Un.manipulac.origen
df10.dropna(subset = 'Un.manipulac.origen', inplace = True)

# ELiminar duplicados
df10.drop_duplicates(subset = 'Un.manipulac.origen', keep = 'first', inplace = True)

# Join
df10.set_index('Un.manipulac.origen', inplace = True)
df1.set_index('UMP.SUP', inplace = True)
df1 = df1.join(df10, how = 'left')
df1.sort_values(by = 'Identificador Unico', ascending = True, inplace = True)
df1.reset_index(inplace = True)
df1.rename(columns = {'index' : 'UMP.SUP'}, inplace = True)

# Cruce Fecha
df1.loc[pd.isna(df1['Status de entrada en stock']), 'Status de entrada en stock'] = \
df1.loc[pd.isna(df1['Status de entrada en stock']), 'Fecha de creación']

# Cruce Hora
df1.loc[pd.isna(df1['Status de la entrada de mercancías']), 'Status de la entrada de mercancías'] = \
df1.loc[pd.isna(df1['Status de la entrada de mercancías']), 'Hora de creación']

# Eliminar columnas del cruce Fecha de creación y Hora de creación
df1.drop(labels = ['Fecha de creación', 'Hora de creación'], axis = 1, inplace = True)

C:\Users\pablo.gutierrez\AppData\Local\Temp\ipykernel_13420\1065728832.py:37: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df1.loc[pd.isna(df1['Status de la entrada de mercancías']), 'Status de la entrada de mercancías'] = \


In [19]:
# Cruce Fecha y Hora con On Hand día anterior

# Cargar On Hand dia anterior
df11 = pd.read_excel(d9030_anterior, dtype = 'string')
df11['D+I'] = df11['Documento'] + df11['Número de posición']

# De aqui saldrá df12 para el cruce de Fecha y Hora, y df13 , para el cruce de Orden de Servicio y Operacion
df12 = df11.loc[: , ['D+I', 'Status de entrada en stock', 'Status de la entrada de mercancías']]

# Renombrar columnas para no tener problemas con el cruce
df12.rename(columns = {'Status de entrada en stock' : 'Status de entrada en stock_2' , 
                       'Status de la entrada de mercancías' : 'Status de la entrada de mercancías_2'}, inplace = True)

# Eliminar duplicados, mantener los primeros
df12.drop_duplicates('D+I', keep = 'first', inplace = True)

# Join
df12.set_index('D+I', inplace = True)
df1.set_index('D+I', inplace = True)
df1 = df1.join(df12, how = 'left')
df1.sort_values(by = 'Identificador Unico', ascending = True, inplace = True)
df1.reset_index(inplace = True)

# Reemplazar celdas vacías Fecha
df1.loc[pd.isna(df1['Status de entrada en stock']), 'Status de entrada en stock'] =\
df1.loc[pd.isna(df1['Status de entrada en stock']), 'Status de entrada en stock_2']

# Reemplazar celdas vacías Hora
df1.loc[pd.isna(df1['Status de la entrada de mercancías']), 'Status de la entrada de mercancías'] =\
df1.loc[pd.isna(df1['Status de la entrada de mercancías']), 'Status de la entrada de mercancías_2']

# Eliminar columnas de cruce
df1.drop(labels = ['Status de entrada en stock_2', 'Status de la entrada de mercancías_2'], axis = 1, inplace = True)

In [20]:
# Cruce Orden y Operacion con el dia anterior, crearemos df13
df13 = df11.loc[: , ['D+I', 'Orden de servicio', 'Operación de orden de servicio']]

# Renombrar columnas, para no tener problemas con el cruce
df13.rename(columns = {'Orden de servicio' : 'Orden de servicio_2', 
                       'Operación de orden de servicio' : 'Operación de orden de servicio_2'}, inplace = True)

# El archivo del día anterior no debería tener valores vacíos en OS y OP, por eso no aplicamos la funcion DropNa

# Eliminar duplicados D+I
df13.drop_duplicates('D+I', keep = 'first', inplace = True)

# Join
df13.set_index('D+I', inplace = True)
df1.set_index('D+I', inplace = True)
df1 = df1.join(df13, how = 'left')
df1.sort_values(by = 'Identificador Unico', ascending = True, inplace = True)
df1.reset_index(inplace = True)

# Rellenar Ordenes de Servicio vacías
df1.loc[pd.isna(df1['Orden de servicio']), 'Orden de servicio'] = \
df1.loc[pd.isna(df1['Orden de servicio']), 'Orden de servicio_2']

# Rellenar Operaciones vacías
df1.loc[pd.isna(df1['Operación de orden de servicio']), 'Operación de orden de servicio'] = \
df1.loc[pd.isna(df1['Operación de orden de servicio']), 'Operación de orden de servicio_2']

# Eliminar columnas del cruce
df1.drop(labels = ['Orden de servicio_2', 'Operación de orden de servicio_2'], axis = 1, inplace = True)

In [21]:
# Todos los propietarios que no pertenezcan al centro de servicios, OS = OP = 0

# Orden de Servicio
df1.loc[((df1['Propietario del stock'] != 'Y5099') & (df1['Propietario del stock'] != 'Y5082') &\
        (df1['Propietario del stock'] != 'Y5059') & (df1['Propietario del stock'] != 'Y5028')) &\
        (pd.isna(df1['Orden de servicio'])) , 'Orden de servicio']  = '0'

# Operación
df1.loc[((df1['Propietario del stock'] != 'Y5099') & (df1['Propietario del stock'] != 'Y5082') &\
        (df1['Propietario del stock'] != 'Y5059') & (df1['Propietario del stock'] != 'Y5028')) &\
        (pd.isna(df1['Operación de orden de servicio'])) , 'Operación de orden de servicio']  = '0'

In [22]:
# Cruce TU
df_TU = pd.read_excel(TU, dtype = 'string', skipfooter = 1,
                      usecols = ['Documento', 'Emisor de mercancía', 'Creados el', 'Unidad de transporte'])

# Arreglar formato Fecha
df_TU['Creados el'] = df_TU['Creados el'].str.replace(" 00:00:00","", regex = False)
df_TU['Creados el'] = pd.to_datetime(df_TU['Creados el'])

# Ordenar fecha según mas reciente
df_TU.sort_values(by = 'Creados el', ascending = False, inplace = True)

# Fecha a dtype string
df_TU['Creados el'] = df_TU['Creados el'].dt.strftime(date_format = '%d/%m/%Y')

# Eliminar Duplicados de documento. Nos quedamos con el mas reciente
df_TU.drop_duplicates(subset = 'Documento', inplace = True)

# Renombrar columnas
df_TU.rename(columns = {'Emisor de mercancía' : 'CENTRO. SUMNISTRO', 'Creados el' : 'FCH. Creacion TU', 
                        'Unidad de transporte' : 'TU' }, inplace = True)

# Documento como Key value para cruce con df1
df_TU.set_index('Documento', inplace = True)

# Documento como Key value en df1
df1.set_index('Documento', inplace = True)

# Join
df1 = df1.join(df_TU, how = 'left')
df1.reset_index(inplace = True)
df1.sort_values(by = 'Identificador Unico', ascending = True, inplace = True)

In [23]:
# Eliminar Columna Identificador Único
df1.drop(labels = 'Identificador Unico', axis = 1, inplace = True)

# Reordenar columnas
df1 = df1[['Documento', 'Número de posición', 'Pedido de compras', 'Posición de pedido', 'Orden de servicio',
           'Operación de orden de servicio', 'Documento ERP','Posición de documento de ERP', 'Aviso de entrega',
           'UMP.SUP', 'UMP', 'Prod.', 'Descripción de producto','Ctd.', 'Unidad medida', 'Propietario del stock',
           'Persona autorizada a disponer', 'Status de entrada en stock', 'Status de la entrada de mercancías',
           'TU', 'FCH. Creacion TU', 'CENTRO. SUMNISTRO']]

# ERP+POS+MAT+CTD como primera columna
df1.set_index('Documento', inplace = True)

In [24]:
# Descargar Archivo
today = date.today()
fecha_hoy = today.strftime("%d-%m-%Y")
Nombre_Archivo = '9030-DESCARGA-' + fecha_hoy + '.xlsx'
df1.to_excel(Nombre_Archivo)

In [25]:
# Abrir excel y modificarlo

# Abrir Instancia Excel
xlApp = win32.Dispatch('Excel.Application')
xlApp.Visible = True

# Abrir 9030-DESCARGA (Libro)
path = os.path.abspath(Nombre_Archivo)
wb = xlApp.Workbooks.Open(path)

# Ajustar Columnas
wb.Worksheets(1).Activate()
wb.ActiveSheet.Columns.AutoFit()

True

In [26]:
# Calculo Tiempo Procesamiento
end_time = time.time()
total_time = end_time - start_time
total_time = total_time / 60
total_time = "{:.2f}".format(total_time)
total_time = str(total_time)
print('Tiempo Total: ' + total_time + ' minutos')

Tiempo Total: 0.15 minutos
